In [ ]:
# Cleaned vs Dirty 

# За основу решения взята baseline от авторов курса 

In [ ]:
import os
import cv2
import time
import copy
import torch
import random
import joblib
import shutil
import zipfile
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms.functional as transf

from PIL import Image
from tqdm import tqdm
from torchvision import datasets, transforms, models

In [ ]:
# Распаковка и создание train и val директорий для дальнейшего формирования датасетов  
with zipfile.ZipFile('../input/platesv2/plates.zip', 'r') as zip_obj:
    zip_obj.extractall('/kaggle/working/')

data_root = '/kaggle/working/plates/'

train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

for dir_name in ['train', 'val']:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

# Распределение на train (16 img) и val (4 img) выборку для каждого из классов 'cleaned', 'dirty'     
for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    names = os.listdir(source_dir)
    names.remove('.DS_Store')
    for i, file_name in enumerate(names):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [ ]:
# Блок определения аугментаций и формирования train и val датасетов

# Список применяемых аугментаций:
#     - center_crop = img вырезается из центра исходника, size = (224,224) 
#     - resize      = исходник сжимается до size = (224,224) 
#     - Hflip       = горизонтальное отзеркаливание  
#     - Vflip       = вертикальное отзеркаливание  
#     - canny       = применение детектора Canny с наложением на img полученных граней 
# Аугментации определены через класс TF_data.

# Состав train датасета (128 img):
#      [0] 16 dirty img + 16 cleaned img, аугментации - center_crop, canny                                      
#      [1] 16 dirty img + 16 cleaned img, аугментации - center_crop, Hflip, canny
#      [2] 16 dirty img + 16 cleaned img, аугментации - center_crop, Vflip, canny
#      [3] 16 dirty img + 16 cleaned img, аугментации - center_crop
                                       
# Состав val датасета (8 img):
#      [0] 4 dirty img + 4 cleaned img, аугментации - resize, canny

# batch_size принят равным 4 (при увеличении наблюдалось снижение accuracy). 
# Итоговый размер train_dataloader - 32 батча из 128 img

mean_ = [0.485, 0.456, 0.406]
std_ = [0.229, 0.224, 0.225]

class TF_data(): 
    def __init__(self, center_crop=False, resize=False, Hflip=False, Vflip=False, canny=False):
        self.center_crop = center_crop
        self.resize = resize
        self.Hflip = Hflip
        self.Vflip = Vflip
        self.canny = canny
    
    def __call__(self, img):
        if self.resize:
            img = transf.resize(img, (224,224))
        if self.center_crop:
            img = transf.center_crop(img, 224)
        if self.Hflip:
            img = transf.hflip(img)
        if self.Vflip:
            img = transf.vflip(img)
        if self.canny:
            im = np.asarray(img)
            edges = cv2.Canny(im,90,180)
            edges = cv2.cvtColor(edges,cv2.COLOR_GRAY2RGB)
            res = cv2.addWeighted(im,0.7,edges,0.3,0)
            img = Image.fromarray(res)

        img = transf.normalize(transf.to_tensor(img), mean_, std_)                 
        return img

set_data = [0 for i in range(4)]

set_data[0] = datasets.ImageFolder(train_dir, TF_data(center_crop=True, canny=True))
set_data[1] = datasets.ImageFolder(train_dir, TF_data(center_crop=True, Hflip=True, canny=True))
set_data[2] = datasets.ImageFolder(train_dir, TF_data(center_crop=True, Vflip=True, canny=True))
set_data[3] = datasets.ImageFolder(train_dir, TF_data(center_crop=True))

train_dataset = torch.utils.data.ConcatDataset(set_data)

val_transforms = TF_data(resize=True, canny=True)
val_dataset = datasets.ImageFolder(val_dir, val_transforms)

batch_size = 4
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [ ]:
len(train_dataloader), len(train_dataset)

In [ ]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def show_input(input_tensor, title=''):
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [ ]:
# Функция режима тренировки модели. Относительно baseline внесено одно изменение - 
# - scheduler.step() выполняется после optimizer.step() согласно рекомендации PyTorch Doc

def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                model.train()
            else:
                dataloader = val_dataloader
                model.eval()

            running_loss = 0.
            running_acc = 0.

            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()
                
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)
            
            train_loss_history[phase].append(epoch_loss)
            train_acc_history[phase].append(epoch_acc)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)
            
    return model

In [ ]:
# Задача классификации решалась с использованием обученных моделей из библиотеки PyTorch.  
# В результате нескольких экспериментов была выбрана модель mobilenet_v2. 
# Базовый классификатор модели был изменен добавлением дополнительных полносвязных слоев с функцией активации ReLU6 и 
# дропаутом. Проводились эксперименты  с постепенным увеличением Linear-слоев с 1-го до 3-х. 
# Увеличение количества полносвязных слоев дало увеличение accuracy с 0.93817 (на 10 эпохах обучения) до 0.96370 (на 30 эпохах обучения)

In [ ]:
# Загрузка обученной модели с измененным классификатором  

def reload():
    model = models.mobilenet_v2(pretrained=True)
    
    i = model.classifier[1].in_features
    m = model.classifier[1].in_features // 3

    model.classifier = torch.nn.Sequential(
                                          torch.nn.Dropout(0.2),
                                          torch.nn.Linear(i, m),        # 1280 --> 426 нейронов
                                          torch.nn.ReLU6(),
                                          torch.nn.Linear(m, m // 2 ),  # 426 --> 213 нейронов
                                          torch.nn.ReLU6(),
                                          torch.nn.Dropout(0.2),
                                          torch.nn.Linear(m // 2, 2)    # 213 --> 2 нейрона
                                          )
    return model

In [ ]:
# Обучение модели
# Идея получения наилучшего результата заключается в выборе best_model путем перебора 10-и seed'ов (history) и 
# сравниванием значений mean_loss и mean_acc для каждой обученной модели. 
# Чем выше acc и меньше loss (для моделей с одинаковым mean_acc), тем предпочтительней модель.
# Модель обучалась 30 эпох с понижением скорости обучения на каждой эпохе в 2 раза.

torch.backends.cudnn.deterministic = True

history = 10
num_epochs = 30
mean_acc = [0 for i in range(history)]
mean_loss = [1 for i in range(history)]

lr = 0.002
momentum = 0.9
weight_decay = 0.1
step_size = 1
gamma = 0.5

for i in range(history):
    random.seed(i)               
    np.random.seed(i)
    torch.manual_seed(i)
    torch.cuda.manual_seed(i)
    
    train_loss_history = {'train':[], 'val':[]}
    train_acc_history = {'train':[], 'val':[]}

    model = reload()  # загрузка модели
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    
    train_model(model, loss, optimizer, scheduler, num_epochs=num_epochs); # обучение модели
    
    mean_loss[i] = sum(train_loss_history['val'])/len(train_loss_history['val'])
    mean_acc[i] = sum(train_acc_history['val'])/len(train_acc_history['val'])
    
    # сохранение best_model, статистик и seed
    if mean_acc[:i] == [] or \
       mean_acc[i] > max(mean_acc[:i]) or \
      (mean_acc[i] == max(mean_acc[:i]) and mean_loss[i] < min(mean_loss[:i])):
            
        best_model = model
        best_seed = i
        best_train_loss_history = train_loss_history
        best_train_acc_history = train_acc_history 
        
    print('history: {:.1f} mean_loss: {:.4f} mean_acc: {:.4f}'.format(i, mean_loss[i], mean_acc[i]), flush=True)
    
print('best_history: {:.1f} mean_loss: {:.4f} mean_acc: {:.4f}'.format(best_seed, mean_loss[best_seed], mean_acc[best_seed]), flush=True)

In [ ]:
# Вывод графика по acc
plt.plot(best_train_acc_history['train'], label='train')
plt.plot(best_train_acc_history['val'], label='val')
plt.legend(loc='lower right')
plt.xlabel('$Epoch$')
plt.ylabel('$Acc$')

In [ ]:
# Вывод графика по loss
plt.plot(best_train_loss_history['train'], label='train')
plt.plot(best_train_loss_history['val'], label='val')
plt.legend(loc='upper right')
plt.xlabel('$Epoch$')
plt.ylabel('$Loss$')

In [ ]:
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [ ]:
# Формирование test_dataset и test_dataloader. 
# Для test_dataset применяются аугментации resize и canny

class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
# Переключение best_model в eval режим и составление predictions для test выборки

best_model.eval()

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = best_model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

In [ ]:
inputs, labels, paths = next(iter(test_dataloader))

for img, pred in zip(inputs, test_predictions):
    show_input(img, title=pred)

In [ ]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.58 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=40)

In [ ]:
submission_df.to_csv('submission.csv')

In [ ]:
# Сохранение модели в файл

filename = '/kaggle/working/best_model.sav'
joblib.dump(best_model, filename)

In [ ]:
!rm -rf train val test